# Chapter 5 - Exercise 4: Mechanical Stimulation Protocol Designer

## Learning Objectives

After completing this exercise, you will be able to:

1. **Design** tissue-specific mechanical conditioning protocols
2. **Understand** how mechanical cues guide tissue maturation
3. **Apply** physiological loading patterns to bioreactor systems
4. **Optimize** stimulation parameters (frequency, magnitude, duration)
5. **Predict** tissue responses to mechanical conditioning

---

## Background: Mechanobiology in Tissue Engineering

### Why Mechanical Stimulation?

In vivo, tissues experience continuous mechanical forces that guide their development, maintenance, and function:

- **Bone**: Compression and shear from weight-bearing (Wolff's law)
- **Cartilage**: Intermittent compression during movement
- **Heart/Muscle**: Cyclic stretch during contraction
- **Blood vessels**: Pulsatile pressure and shear flow
- **Tendon/Ligament**: Uniaxial tension

**Mechanical conditioning in bioreactors reproduces these forces to:**
- Promote cell alignment and organization
- Enhance ECM production and remodeling
- Increase tissue mechanical strength
- Guide differentiation and maturation

### Types of Mechanical Stimulation

#### 1. Compression
**Application:** Bone, cartilage  
**Mechanism:** Hydrostatic pressure, fluid flow through pores  
**Parameters:**
- Strain magnitude: 1-15%
- Frequency: 0.1-2 Hz
- Duration: 0.5-4 hours/day

#### 2. Tension/Stretch
**Application:** Heart, muscle, tendon, blood vessels  
**Mechanism:** Cell alignment, cytoskeletal reorganization  
**Parameters:**
- Strain magnitude: 5-15%
- Frequency: 0.5-1.5 Hz (physiological heart rate)
- Duration: Continuous or cyclic

#### 3. Shear Stress
**Application:** Blood vessels (endothelium), bone (canaliculi)  
**Mechanism:** Mechanotransduction via glycocalyx, ion channels  
**Parameters:**
- Magnitude: 0.5-20 dyne/cm² (5-200 mPa)
- Pattern: Steady laminar or pulsatile
- Duration: Hours to continuous

#### 4. Hydrostatic Pressure
**Application:** Cartilage, intervertebral disc  
**Mechanism:** Nutrient transport, chondrocyte metabolism  
**Parameters:**
- Magnitude: 0.1-10 MPa
- Frequency: 0.1-1 Hz
- Duration: 1-4 hours/day

---

## Key Concepts

### Mechanotransduction
**Process:** Mechanical stimulus → cellular response

**Pathway:**
1. **Mechanosensors**: Integrins, ion channels, primary cilia
2. **Signal transduction**: FAK, MAPK, Ca²⁺ signaling
3. **Gene expression**: ECM proteins, growth factors
4. **Tissue adaptation**: Increased strength, organization

### Dose-Response Relationship

$$\text{Tissue Response} = f(\text{Magnitude}, \text{Frequency}, \text{Duration})$$

**Optimal window:**
- Too low → Insufficient stimulus (disuse atrophy)
- Optimal → Maximal adaptation
- Too high → Damage, inflammation

### Physiological Loading Patterns

| Tissue | In Vivo Loading | Bioreactor Analog |
|--------|----------------|------------------|
| Heart | 60-80 bpm, 10-15% strain | Cyclic stretch, 1 Hz |
| Bone | Walking: 1-2 Hz, 0.1-0.3% strain | Compression bioreactor |
| Cartilage | Intermittent load, 15% strain | Dynamic compression |
| Artery | Pulsatile, 80 bpm, 10-15 dyne/cm² | Flow loop, pulsatile pump |
| Tendon | High tension, low frequency | Uniaxial stretch |

---

## Setup: Import Required Libraries

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact, widgets, Layout
from IPython.display import display, HTML
import seaborn as sns
from matplotlib.patches import Rectangle

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Libraries imported successfully!")
print("Ready to design mechanical stimulation protocols...")

## Tissue Parameters Database

In [ ]:
# Tissue-specific mechanical stimulation parameters
TISSUE_PARAMS = {
    'Bone': {
        'primary_stimulus': 'Compression',
        'strain_range': (0.1, 3.0),  # % strain
        'frequency_range': (0.5, 2.0),  # Hz
        'optimal_strain': 1.0,
        'optimal_frequency': 1.0,
        'duration': 2.0,  # hours/day
        'rest_periods': True,
        'native_modulus': 18000,  # MPa
        'description': 'Osteogenic differentiation, mineralization',
        'maturation_time': 14,  # days
    },
    'Cartilage': {
        'primary_stimulus': 'Dynamic Compression',
        'strain_range': (5, 15),
        'frequency_range': (0.1, 1.0),
        'optimal_strain': 10.0,
        'optimal_frequency': 0.5,
        'duration': 3.0,
        'rest_periods': True,
        'native_modulus': 0.8,  # MPa
        'description': 'Chondrogenic differentiation, GAG production',
        'maturation_time': 21,
    },
    'Cardiac Tissue': {
        'primary_stimulus': 'Cyclic Stretch',
        'strain_range': (5, 20),
        'frequency_range': (0.5, 2.0),
        'optimal_strain': 12.0,
        'optimal_frequency': 1.0,  # 60 bpm
        'duration': 24.0,  # Continuous
        'rest_periods': False,
        'native_modulus': 0.05,  # MPa (anisotropic)
        'description': 'Sarcomere alignment, contractile function',
        'maturation_time': 14,
    },
    'Skeletal Muscle': {
        'primary_stimulus': 'Uniaxial Stretch',
        'strain_range': (5, 15),
        'frequency_range': (0.1, 1.0),
        'optimal_strain': 10.0,
        'optimal_frequency': 0.5,
        'duration': 4.0,
        'rest_periods': True,
        'native_modulus': 0.15,  # MPa
        'description': 'Myotube formation, force generation',
        'maturation_time': 10,
    },
    'Blood Vessel': {
        'primary_stimulus': 'Shear Stress + Cyclic Strain',
        'strain_range': (5, 15),
        'frequency_range': (0.8, 1.5),  # 50-90 bpm
        'optimal_strain': 10.0,
        'optimal_frequency': 1.2,  # 72 bpm
        'shear_stress': 15,  # dyne/cm²
        'duration': 24.0,  # Continuous flow
        'rest_periods': False,
        'native_modulus': 1.5,  # MPa
        'description': 'Endothelial alignment, SMC organization',
        'maturation_time': 7,
    },
    'Tendon/Ligament': {
        'primary_stimulus': 'Uniaxial Tension',
        'strain_range': (2, 8),
        'frequency_range': (0.1, 0.5),
        'optimal_strain': 4.0,
        'optimal_frequency': 0.25,
        'duration': 2.0,
        'rest_periods': True,
        'native_modulus': 500,  # MPa
        'description': 'Collagen fiber alignment, tensile strength',
        'maturation_time': 21,
    },
    'Skin': {
        'primary_stimulus': 'Biaxial Stretch',
        'strain_range': (5, 20),
        'frequency_range': (0.05, 0.5),
        'optimal_strain': 10.0,
        'optimal_frequency': 0.1,
        'duration': 1.0,
        'rest_periods': True,
        'native_modulus': 0.5,  # MPa (highly variable)
        'description': 'Dermal fibroblast activation, ECM deposition',
        'maturation_time': 14,
    },
}

print("✓ Tissue parameters loaded")
print(f"  - {len(TISSUE_PARAMS)} tissue types")
print("\n📊 Tissue Overview:")
for tissue, params in TISSUE_PARAMS.items():
    print(f"  {tissue:20s}: {params['primary_stimulus']:25s} → {params['description']}")

## Protocol Design Functions

In [ ]:
def generate_loading_pattern(frequency, duration_hours, strain_magnitude, pattern_type='sinusoidal'):
    """
    Generate mechanical loading waveform.
    
    Parameters:
    -----------
    frequency : float
        Loading frequency in Hz
    duration_hours : float
        Total duration in hours
    strain_magnitude : float
        Peak strain in %
    pattern_type : str
        'sinusoidal', 'square', 'triangular', 'physiological'
    """
    
    # Time array (sample at 100 Hz)
    duration_sec = duration_hours * 3600
    sampling_rate = 100  # Hz
    # Limit points for visualization
    max_points = 10000
    if duration_sec * sampling_rate > max_points:
        sampling_rate = max_points / duration_sec
    
    t = np.linspace(0, duration_sec, int(duration_sec * sampling_rate))
    
    if pattern_type == 'sinusoidal':
        strain = strain_magnitude * np.sin(2 * np.pi * frequency * t)
    elif pattern_type == 'square':
        strain = strain_magnitude * np.sign(np.sin(2 * np.pi * frequency * t))
    elif pattern_type == 'triangular':
        strain = strain_magnitude * (2 * np.abs((frequency * t) % 1 - 0.5) * 2 - 1)
    elif pattern_type == 'physiological':
        # Cardiac-like: sharper systole, gentler diastole
        phase = (frequency * t) % 1
        strain = strain_magnitude * (np.sin(2 * np.pi * phase) * 
                                    (1 + 0.3 * np.sin(4 * np.pi * phase)))
    else:
        strain = np.zeros_like(t)
    
    return t, strain


def calculate_maturation_score(strain, frequency, duration, tissue_params):
    """
    Calculate predicted tissue maturation score (0-100).
    
    Based on deviation from optimal parameters.
    """
    
    # Strain score (Gaussian centered on optimal)
    optimal_strain = tissue_params['optimal_strain']
    strain_range = tissue_params['strain_range']
    strain_width = (strain_range[1] - strain_range[0]) / 4
    strain_score = np.exp(-((strain - optimal_strain) / strain_width) ** 2)
    
    # Frequency score
    optimal_freq = tissue_params['optimal_frequency']
    freq_range = tissue_params['frequency_range']
    freq_width = (freq_range[1] - freq_range[0]) / 4
    freq_score = np.exp(-((frequency - optimal_freq) / freq_width) ** 2)
    
    # Duration score (closer to recommended = better)
    optimal_duration = tissue_params['duration']
    duration_score = np.exp(-((duration - optimal_duration) / 2) ** 2)
    
    # Combined score
    total_score = (strain_score * 0.4 + freq_score * 0.4 + duration_score * 0.2) * 100
    
    return total_score, strain_score, freq_score, duration_score


def calculate_mechanical_dose(frequency, duration_hours, strain_magnitude):
    """
    Calculate total mechanical dose.
    
    Dose = number of cycles × strain magnitude
    """
    
    total_cycles = frequency * duration_hours * 3600
    mechanical_dose = total_cycles * strain_magnitude
    
    return total_cycles, mechanical_dose


def estimate_ecm_production(maturation_score, duration_days, tissue_params):
    """
    Estimate ECM production and tissue stiffness over culture period.
    """
    
    days = np.linspace(0, duration_days, 100)
    
    # Logistic growth model for ECM accumulation
    # Rate depends on maturation score
    k = 0.3 * (maturation_score / 100)  # Growth rate
    t_half = tissue_params['maturation_time']  # Half-saturation time
    
    ecm_fraction = 1 / (1 + np.exp(-k * (days - t_half)))
    
    # Mechanical stiffness follows ECM accumulation
    native_modulus = tissue_params['native_modulus']
    modulus = ecm_fraction * native_modulus * (maturation_score / 100)
    
    return days, ecm_fraction, modulus

print("✓ Protocol design functions defined")

---

## Interactive Tool: Mechanical Stimulation Protocol Designer

### Instructions:

1. **Select tissue type** to load physiological parameters
2. **Design your protocol:**
   - Strain magnitude (%)
   - Frequency (Hz)
   - Duration per day (hours)
   - Loading pattern type
3. **Set culture duration** (days for maturation)
4. **Observe:**
   - Loading waveform
   - Maturation score
   - Predicted tissue properties
   - Protocol recommendations

In [ ]:
def interactive_protocol_designer(tissue_type='Cardiac Tissue',
                                 strain_magnitude=12.0,
                                 frequency=1.0,
                                 duration_hours=6.0,
                                 pattern_type='sinusoidal',
                                 culture_days=14):
    """
    Interactive mechanical stimulation protocol designer.
    """
    
    # Get tissue parameters
    params = TISSUE_PARAMS[tissue_type]
    
    # Generate loading pattern
    t, strain = generate_loading_pattern(frequency, duration_hours, strain_magnitude, pattern_type)
    
    # Calculate scores
    maturation_score, strain_score, freq_score, dur_score = calculate_maturation_score(
        strain_magnitude, frequency, duration_hours, params)
    
    # Calculate mechanical dose
    total_cycles, mechanical_dose = calculate_mechanical_dose(frequency, duration_hours, strain_magnitude)
    
    # Estimate tissue maturation
    days, ecm_fraction, modulus = estimate_ecm_production(maturation_score, culture_days, params)
    
    # Create visualizations
    fig = plt.figure(figsize=(18, 12))
    
    # 1. Loading waveform (show first few cycles)
    ax1 = plt.subplot(3, 3, 1)
    display_duration = min(5 / frequency, duration_hours * 3600)  # Show 5 cycles or all
    mask = t <= display_duration
    ax1.plot(t[mask], strain[mask], 'b-', linewidth=2)
    ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax1.axhline(y=strain_magnitude, color='red', linestyle='--', alpha=0.5, label='Peak strain')
    ax1.axhline(y=-strain_magnitude, color='red', linestyle='--', alpha=0.5)
    ax1.set_xlabel('Time (seconds)', fontsize=11, fontweight='bold')
    ax1.set_ylabel('Strain (%)', fontsize=11, fontweight='bold')
    ax1.set_title(f'Loading Pattern: {pattern_type.capitalize()}', fontsize=13, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Optimal parameter ranges
    ax2 = plt.subplot(3, 3, 2)
    
    # Strain range
    strain_range = params['strain_range']
    optimal_strain = params['optimal_strain']
    
    # Create strain response curve
    strain_values = np.linspace(0, strain_range[1] * 1.5, 100)
    strain_responses = [calculate_maturation_score(s, frequency, duration_hours, params)[1] * 100 
                       for s in strain_values]
    
    ax2.plot(strain_values, strain_responses, 'b-', linewidth=2.5, label='Strain response')
    ax2.axvspan(strain_range[0], strain_range[1], alpha=0.2, color='green', label='Physiological range')
    ax2.axvline(x=optimal_strain, color='red', linestyle='--', linewidth=2, label='Optimal')
    ax2.plot(strain_magnitude, strain_score * 100, 'ro', markersize=12, label='Current')
    ax2.set_xlabel('Strain Magnitude (%)', fontsize=11, fontweight='bold')
    ax2.set_ylabel('Response Score', fontsize=11, fontweight='bold')
    ax2.set_title('Strain Dose-Response', fontsize=13, fontweight='bold')
    ax2.legend(loc='best', fontsize=9)
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim([0, 105])
    
    # 3. Frequency response
    ax3 = plt.subplot(3, 3, 3)
    
    freq_range = params['frequency_range']
    optimal_freq = params['optimal_frequency']
    
    freq_values = np.linspace(0, freq_range[1] * 1.5, 100)
    freq_responses = [calculate_maturation_score(strain_magnitude, f, duration_hours, params)[2] * 100 
                     for f in freq_values]
    
    ax3.plot(freq_values, freq_responses, 'g-', linewidth=2.5, label='Frequency response')
    ax3.axvspan(freq_range[0], freq_range[1], alpha=0.2, color='green', label='Physiological range')
    ax3.axvline(x=optimal_freq, color='red', linestyle='--', linewidth=2, label='Optimal')
    ax3.plot(frequency, freq_score * 100, 'ro', markersize=12, label='Current')
    ax3.set_xlabel('Frequency (Hz)', fontsize=11, fontweight='bold')
    ax3.set_ylabel('Response Score', fontsize=11, fontweight='bold')
    ax3.set_title('Frequency Dose-Response', fontsize=13, fontweight='bold')
    ax3.legend(loc='best', fontsize=9)
    ax3.grid(True, alpha=0.3)
    ax3.set_ylim([0, 105])
    
    # 4. Maturation score breakdown
    ax4 = plt.subplot(3, 3, 4)
    
    categories = ['Strain\nMagnitude', 'Frequency', 'Duration', 'Overall\nScore']
    scores = [strain_score * 100, freq_score * 100, dur_score * 100, maturation_score]
    colors = ['#e74c3c' if s < 60 else '#f39c12' if s < 80 else '#27ae60' for s in scores]
    
    bars = ax4.bar(categories, scores, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
    ax4.axhline(y=80, color='green', linestyle='--', label='Excellent (>80)', alpha=0.5)
    ax4.axhline(y=60, color='orange', linestyle='--', label='Good (>60)', alpha=0.5)
    ax4.set_ylabel('Score', fontsize=11, fontweight='bold')
    ax4.set_title('Protocol Optimization Score', fontsize=13, fontweight='bold')
    ax4.set_ylim([0, 105])
    ax4.legend(loc='lower right', fontsize=9)
    ax4.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bar, score in zip(bars, scores):
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height + 2,
                f'{score:.0f}', ha='center', va='bottom', fontweight='bold', fontsize=10)
    
    # 5. ECM accumulation over time
    ax5 = plt.subplot(3, 3, 5)
    ax5.plot(days, ecm_fraction * 100, 'purple', linewidth=2.5, label='ECM Deposition')
    ax5.axhline(y=90, color='green', linestyle='--', alpha=0.5, label='Mature (>90%)')
    ax5.axvline(x=params['maturation_time'], color='red', linestyle='--', 
               alpha=0.5, label=f"t₁/₂ = {params['maturation_time']} days")
    ax5.set_xlabel('Culture Duration (days)', fontsize=11, fontweight='bold')
    ax5.set_ylabel('ECM Deposition (%)', fontsize=11, fontweight='bold')
    ax5.set_title('Tissue Maturation Kinetics', fontsize=13, fontweight='bold')
    ax5.legend(loc='best')
    ax5.grid(True, alpha=0.3)
    ax5.set_ylim([0, 105])
    
    # 6. Mechanical stiffness development
    ax6 = plt.subplot(3, 3, 6)
    ax6.plot(days, modulus, 'brown', linewidth=2.5, label='Young\'s Modulus')
    ax6.axhline(y=params['native_modulus'], color='green', linestyle='--', 
               label=f"Native: {params['native_modulus']} MPa", alpha=0.7)
    ax6.set_xlabel('Culture Duration (days)', fontsize=11, fontweight='bold')
    ax6.set_ylabel('Modulus (MPa)', fontsize=11, fontweight='bold')
    ax6.set_title('Mechanical Property Development', fontsize=13, fontweight='bold')
    ax6.legend(loc='best')
    ax6.grid(True, alpha=0.3)
    
    # 7. Daily loading schedule
    ax7 = plt.subplot(3, 3, 7)
    ax7.axis('off')
    
    # Create daily schedule visualization
    fig_schedule = plt.figure(figsize=(8, 2))
    ax_sch = fig_schedule.add_subplot(111)
    
    hours_in_day = 24
    
    # Draw time blocks
    if params['rest_periods']:
        # Intermittent loading
        ax_sch.add_patch(Rectangle((0, 0), duration_hours, 1, 
                                   facecolor='green', alpha=0.6, label='Loading'))
        ax_sch.add_patch(Rectangle((duration_hours, 0), hours_in_day - duration_hours, 1,
                                   facecolor='lightblue', alpha=0.6, label='Rest'))
    else:
        # Continuous loading
        ax_sch.add_patch(Rectangle((0, 0), hours_in_day, 1, 
                                   facecolor='green', alpha=0.6, label='Continuous'))
    
    ax_sch.set_xlim([0, hours_in_day])
    ax_sch.set_ylim([0, 1])
    ax_sch.set_xlabel('Time (hours)', fontsize=11, fontweight='bold')
    ax_sch.set_yticks([])
    ax_sch.set_title('Daily Loading Schedule', fontsize=12, fontweight='bold')
    ax_sch.legend(loc='upper right')
    ax_sch.set_xticks(np.arange(0, 25, 4))
    ax_sch.grid(axis='x', alpha=0.3)
    
    plt.close(fig_schedule)
    
    # Show in main figure
    from matplotlib.backends.backend_agg import FigureCanvasAgg
    canvas = FigureCanvasAgg(fig_schedule)
    canvas.draw()
    
    # 8. Mechanical dose analysis
    ax8 = plt.subplot(3, 3, 8)
    
    # Compare with different durations
    duration_range = np.linspace(0.5, 8, 50)
    doses = [calculate_mechanical_dose(frequency, d, strain_magnitude)[1] for d in duration_range]
    
    ax8.plot(duration_range, doses, 'orange', linewidth=2.5)
    ax8.plot(duration_hours, mechanical_dose, 'ro', markersize=12, label='Current protocol')
    ax8.axvline(x=params['duration'], color='green', linestyle='--', 
               label=f"Recommended: {params['duration']}h", alpha=0.7)
    ax8.set_xlabel('Daily Duration (hours)', fontsize=11, fontweight='bold')
    ax8.set_ylabel('Mechanical Dose (cycles × % strain)', fontsize=11, fontweight='bold')
    ax8.set_title('Cumulative Mechanical Dose', fontsize=13, fontweight='bold')
    ax8.legend(loc='best')
    ax8.grid(True, alpha=0.3)
    
    # 9. Summary table
    ax9 = plt.subplot(3, 3, 9)
    ax9.axis('off')
    
    final_ecm = ecm_fraction[-1] * 100
    final_modulus = modulus[-1]
    
    summary_text = f"""
    PROTOCOL SUMMARY
    {'='*50}
    
    TISSUE: {tissue_type}
    Application: {params['description']}
    
    STIMULATION PARAMETERS
    Type: {params['primary_stimulus']}
    Strain: {strain_magnitude:.1f}% 
           (optimal: {params['optimal_strain']:.1f}%)
    Frequency: {frequency:.2f} Hz 
              (optimal: {params['optimal_frequency']:.2f} Hz)
    Duration: {duration_hours:.1f} h/day 
             (recommended: {params['duration']:.1f} h)
    Pattern: {pattern_type.capitalize()}
    
    MECHANICAL DOSE
    Cycles/day: {total_cycles:.0f}
    Total dose: {mechanical_dose:.0f} cycle·%strain
    
    PREDICTED OUTCOMES ({culture_days} days)
    Maturation score: {maturation_score:.0f}/100
    ECM deposition: {final_ecm:.0f}%
    Stiffness: {final_modulus:.2f} MPa
              ({final_modulus/params['native_modulus']*100:.0f}% of native)
    
    STATUS: {self._get_status(maturation_score)}
    """
    
    ax9.text(0.05, 0.95, summary_text, transform=ax9.transAxes,
            fontsize=9, verticalalignment='top', family='monospace',
            bbox=dict(boxstyle='round', facecolor='lightgreen' if maturation_score > 80 
                     else 'lightyellow' if maturation_score > 60 else 'lightcoral', alpha=0.5))
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed analysis
    print("\n" + "="*80)
    print("📊 PROTOCOL ANALYSIS")
    print("="*80)
    
    print(f"\n1. PARAMETER OPTIMIZATION")
    print(f"   Strain magnitude: {strain_magnitude:.1f}% ", end="")
    if abs(strain_magnitude - params['optimal_strain']) < 2:
        print("✓ Optimal")
    elif params['strain_range'][0] <= strain_magnitude <= params['strain_range'][1]:
        print("⚠️  Suboptimal but acceptable")
    else:
        print("✗ Outside physiological range")
    
    print(f"   Frequency: {frequency:.2f} Hz ", end="")
    if abs(frequency - params['optimal_frequency']) < 0.2:
        print("✓ Optimal")
    elif params['frequency_range'][0] <= frequency <= params['frequency_range'][1]:
        print("⚠️  Suboptimal but acceptable")
    else:
        print("✗ Outside physiological range")
    
    print(f"   Duration: {duration_hours:.1f} h/day ", end="")
    if abs(duration_hours - params['duration']) < 1:
        print("✓ Optimal")
    else:
        print("⚠️  Consider adjusting")
    
    print(f"\n2. MATURATION PREDICTION")
    print(f"   Overall score: {maturation_score:.0f}/100 - ", end="")
    if maturation_score > 80:
        print("Excellent protocol")
    elif maturation_score > 60:
        print("Good protocol with room for improvement")
    else:
        print("Suboptimal - recommend parameter adjustment")
    
    print(f"   Expected ECM: {final_ecm:.0f}% after {culture_days} days")
    print(f"   Expected modulus: {final_modulus:.2f} MPa ({final_modulus/params['native_modulus']*100:.0f}% of native)")
    
    print(f"\n3. RECOMMENDATIONS")
    
    if maturation_score < 80:
        improvements = []
        if strain_score < 0.9:
            improvements.append(f"Adjust strain to {params['optimal_strain']:.1f}%")
        if freq_score < 0.9:
            improvements.append(f"Adjust frequency to {params['optimal_frequency']:.2f} Hz")
        if dur_score < 0.9:
            improvements.append(f"Adjust duration to {params['duration']:.1f} h/day")
        
        if improvements:
            print("   To improve maturation:")
            for imp in improvements:
                print(f"   → {imp}")
    else:
        print("   ✓ Protocol is well-optimized")
        print("   → Maintain current parameters")
        print("   → Monitor tissue response regularly")
    
    if params['rest_periods']:
        print(f"\n   ⚠️  Remember: This tissue requires rest periods")
        print(f"      Load for {duration_hours:.1f}h, then rest for {24-duration_hours:.1f}h")
    
    print(f"\n4. IMPLEMENTATION NOTES")
    print(f"   → Total cycles per day: {total_cycles:.0f}")
    print(f"   → Pattern type: {pattern_type}")
    print(f"   → Expected maturation time: ~{params['maturation_time']} days")
    
    print("\n" + "="*80)

# Helper method for status
def _get_status(score):
    if score > 80:
        return "✓ EXCELLENT"
    elif score > 60:
        return "⚠️ GOOD"
    else:
        return "✗ NEEDS OPTIMIZATION"

print("✓ Interactive protocol designer ready")

---

## 🎮 INTERACTIVE TOOL: Run This Cell!

In [ ]:
interact(interactive_protocol_designer,
         tissue_type=widgets.Dropdown(
             options=list(TISSUE_PARAMS.keys()),
             value='Cardiac Tissue',
             description='Tissue Type:',
             style={'description_width': '150px'},
             layout=Layout(width='400px')
         ),
         strain_magnitude=widgets.FloatSlider(
             value=12.0,
             min=0.5,
             max=25.0,
             step=0.5,
             description='Strain Magnitude (%):',
             style={'description_width': '150px'},
             layout=Layout(width='550px')
         ),
         frequency=widgets.FloatSlider(
             value=1.0,
             min=0.05,
             max=2.5,
             step=0.05,
             description='Frequency (Hz):',
             style={'description_width': '150px'},
             layout=Layout(width='550px')
         ),
         duration_hours=widgets.FloatSlider(
             value=6.0,
             min=0.5,
             max=24.0,
             step=0.5,
             description='Duration (h/day):',
             style={'description_width': '150px'},
             layout=Layout(width='550px')
         ),
         pattern_type=widgets.Dropdown(
             options=['sinusoidal', 'square', 'triangular', 'physiological'],
             value='sinusoidal',
             description='Loading Pattern:',
             style={'description_width': '150px'},
             layout=Layout(width='400px')
         ),
         culture_days=widgets.IntSlider(
             value=14,
             min=3,
             max=42,
             step=1,
             description='Culture Duration (days):',
             style={'description_width': '150px'},
             layout=Layout(width='550px')
         ));

---

## 📝 Student Exercises

### Exercise 1: Tissue-Specific Optimization
**Design optimal protocols for each tissue type:**

1. **Bone**: Find parameters that achieve >80 maturation score
2. **Cardiac Tissue**: Match physiological heart rate (60-80 bpm = 1-1.33 Hz)
3. **Cartilage**: Balance strain magnitude to avoid damage but promote GAG production
4. Compare final stiffness achieved for each tissue type

### Exercise 2: Parameter Sensitivity
**Using Cardiac Tissue:**

1. Start with optimal parameters (12% strain, 1 Hz, 6h). Note maturation score.
2. Vary ONLY strain (5%, 12%, 20%). Which parameter matters most?
3. Vary ONLY frequency (0.5, 1.0, 2.0 Hz). How sensitive is the tissue?
4. Which parameter has the strongest effect on maturation?

### Exercise 3: Loading Pattern Comparison
**Using Skeletal Muscle at 10% strain, 0.5 Hz, 4h:**

1. Compare all 4 loading patterns (sinusoidal, square, triangular, physiological)
2. Does pattern type significantly affect maturation score?
3. Which pattern best mimics natural muscle contraction?
4. When might square waves cause cell damage?

### Exercise 4: Duration vs Intensity Trade-off
**Using Bone tissue:**

1. Protocol A: 3% strain, 1 Hz, 1 hour/day
2. Protocol B: 1% strain, 1 Hz, 3 hours/day
3. Calculate mechanical dose for each
4. Which achieves better maturation? Why?
5. What are practical advantages of each approach?

### Exercise 5: Maturation Timeline
**Compare maturation rates across tissue types:**

1. Use optimal parameters for each tissue
2. Set culture duration to 21 days for all
3. Which tissue matures fastest? Slowest?
4. When does each tissue reach 90% ECM deposition?
5. How does this relate to clinical translation timelines?

### Exercise 6: Suboptimal Protocol Analysis
**Intentionally create poor protocols:**

1. Cardiac tissue: Use 25% strain, 2.5 Hz (extreme conditions)
2. What is the maturation score?
3. What problems would this cause? (Consider cell damage)
4. How would you fix this protocol?

### Exercise 7: Clinical Application
**Design protocols for specific applications:**

1. **Cartilage repair**: Need mechanically strong construct in 3 weeks
2. **Heart patch**: Need contractile tissue for transplantation
3. **Vascular graft**: Need burst pressure >120 mmHg
4. What parameters would you choose for each?

---

## 💭 Discussion Questions

1. **Mechanotransduction Pathways:**
   - How do cells sense mechanical forces?
   - Why do different cell types respond differently to the same stimulus?

2. **Dose-Response Relationships:**
   - Why is there an optimal window for mechanical stimulation?
   - What happens with too little stimulation? Too much?

3. **Rest Periods:**
   - Why do bone and cartilage need rest periods?
   - Why is cardiac tissue loaded continuously?

4. **Pattern vs Dose:**
   - Is total mechanical dose more important than pattern?
   - When does waveform shape matter?

5. **Translation Challenges:**
   - How do you scale mechanical stimulation from research to production?
   - What are the engineering challenges in building stimulation bioreactors?

6. **Multi-Modal Stimulation:**
   - Should you combine mechanical + electrical stimulation?
   - How do biochemical factors interact with mechanical cues?

---

## 📚 Key Takeaways

✅ **Mechanical forces guide tissue development** through mechanotransduction pathways

✅ **Each tissue has optimal loading conditions:**
- Bone: 1% strain, 1 Hz, intermittent
- Cartilage: 10% strain, 0.5 Hz, intermittent  
- Cardiac: 12% strain, 1 Hz, continuous
- Vascular: 10% strain + 15 dyne/cm² shear, continuous

✅ **Protocol parameters:**
- **Strain magnitude**: Peak deformation (%)
- **Frequency**: Loading cycles per second (Hz)
- **Duration**: Hours of stimulation per day
- **Pattern**: Waveform shape (sinusoidal, square, etc.)

✅ **Dose-response relationship:**
$$\text{Maturation} = f(\text{Strain} \times \text{Frequency} \times \text{Duration})$$

✅ **Rest periods are critical** for:
- Bone and cartilage (anabolic response)
- Avoiding overload damage
- Nutrient exchange

✅ **Continuous loading needed** for:
- Cardiac tissue (maintains contractile phenotype)
- Vascular tissue (endothelial alignment)

✅ **Maturation outcomes:**
- Enhanced ECM deposition
- Increased mechanical strength
- Cell alignment and organization
- Functional tissue properties

✅ **Practical considerations:**
- Start conservative, increase gradually
- Monitor tissue response (viability, gene expression)
- Combine with proper nutrient supply (Exercise 2)
- Match bioreactor capabilities (Exercise 1)

---

## 🔗 Connection to Chapter 5

This exercise connects to:
- **Section 5.3.2:** Physical stimuli in bioreactors
- **Section 5.5.1:** Principles of tissue maturation
- **Figure 5.6:** Physical stimuli types
- **Table (tissue types):** Stimulus-specific requirements

Related exercises:
- **Exercise 1:** Bioreactor selection (which reactor provides which stimulation?)
- **Exercise 2:** Oxygen transfer (increased with mechanical conditioning)
- **Exercise 5:** Perfusion systems (nutrient delivery during stimulation)
- **Exercise 7:** Maturation monitoring (assessing protocol effectiveness)

---

*Developed for Master's-level Bioengineering students*  
*Vrije Universiteit Brussel - Biofabrication Course 2025*